<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/Automatic_Differentiation_With_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [10]:
x = torch.ones(5) #Input tensor
y = torch.zeros(3) #Expected output 
w = torch.randn(5,3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x,w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y) #Calculate the loss,  distance between the predicted and the actual or desired output 
#L(y, ^y) 

In [11]:
x = torch.zeros(5) #Input tensor 
y = torch.ones(3) #Expected output
w = torch.randn(5,3, requires_grad = True)
b = torch.randn(3,requires_grad= True)
z = torch.matmul(x,w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [12]:
#Tensor Functions and Computational graphs